In [1]:
# Import packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn Packages
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Sklearn Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error, precision_score, confusion_matrix, accuracy_score

# Visualizes all the columns
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Import dataset
#path = '/content/Exoplanet_ml'

df = pd.read_csv('cleaned.csv')


In [3]:
features = df.drop(columns=['row','kepid','koi_disposition','koi_score','ExoplanetCandidate'])
target = df.ExoplanetCandidate

In [4]:
df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1, test_size=.30)

In [6]:
# Evaluation function

def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('Recall: ' + str(metrics.recall_score(y_test, y_pred)))
    print('F1 Score: ' + str(metrics.f1_score(y_test, y_pred)))
    print('Precision: ' + str(metrics.precision_score(y_test, y_pred)))
    
# Print Confusion Matrix
    print('\nConfusion Matrix:')
    print(' TN,  FP, FN, TP')
    print(confusion_matrix(y_true, y_pred).ravel())
    
# Function Prints best parameters for GridSearchCV
def print_results(results):
    print('Best Parameters: {}\n'.format(results.best_params_)) 
    

In [7]:
# Logistic Regression Model
lr = LogisticRegression(C=100, max_iter=200, class_weight='balanced')

# Fitting Model to the train set
lr.fit(X_train, y_train)

# Predicting on the test set
y_pred = lr.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.5849858356940509
Recall: 0.6760299625468165
F1 Score: 0.7113300492610837
Precision: 0.7505197505197505

Confusion Matrix:
 TN,  FP, FN, TP
[ 52 120 173 361]


In [8]:
knn = KNeighborsClassifier(leaf_size=8, metric='manhattan',weights='uniform')

# Fitting Model to the train set
knn.fit(X_train, y_train)

# Predicting on the test set
y_pred = knn.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.7082152974504249
Recall: 0.9063670411985019
F1 Score: 0.8245315161839863
Precision: 0.75625

Confusion Matrix:
 TN,  FP, FN, TP
[ 16 156  50 484]


In [9]:
tree = DecisionTreeClassifier()

# Fitting Model to the train set
tree.fit(X_train, y_train)

# Predicting on the test set
y_pred = tree.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.6614730878186968
Recall: 0.7827715355805244
F1 Score: 0.7776744186046511
Precision: 0.7726432532347505

Confusion Matrix:
 TN,  FP, FN, TP
[ 49 123 116 418]


In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm.notebook import tqdm
from time import sleep

In [11]:
parameter_n_estimators = [100]
for i in tqdm(parameter_n_estimators):
    # Instantiate model
    forest = RandomForestClassifier(n_estimators=i, criterion='gini')
    # Fitting Model to the train set
    forest.fit(X_train, y_train)
    # Predicting on the test set
    y_pred = forest.predict(X_test)

    # Evaluating model
    evaluation(y_test, y_pred)
    print('Tree: %s ' % (i))

  0%|          | 0/1 [00:00<?, ?it/s]

Evaluation Metrics:
Accuracy: 0.6869688385269122
Recall: 0.8426966292134831
F1 Score: 0.8028545941123997
Precision: 0.7666098807495741

Confusion Matrix:
 TN,  FP, FN, TP
[ 35 137  84 450]
Tree: 100 


In [12]:
import joblib

In [13]:
#joblib.dump(forest, 'ml_model2.joblib')

In [16]:
X_test.tail()

flux   Unnamed: 5   Unnamed: 6   Unnamed: 7   Unnamed: 8  \
1765  73243.773438  73282.14063  73289.52344  73280.83594  73290.84375   
1813  10804.450195  10809.66016  10818.52539  10809.08203  10805.33594   
1355  22160.751953  22156.48828  22162.75781  22152.43945  22157.66406   
1509  29327.216797  29317.24609  29318.05859  29317.74414  29312.93555   
2164  19585.585938  19572.84375  19580.64844  19600.86914  19591.43555   

       Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  \
1765  73275.07813  73283.85938  73288.71094  73280.53906  73281.48438   
1813  10813.56641  10798.83594  10807.15430  10794.94043  10795.88965   
1355  22150.48438  22162.16602  22154.97852  22154.32422  22159.79297   
1509  29317.22656  29310.33398  29333.07227  29322.75586  29323.70313   
2164  19597.98828  19604.46680  19614.05664  19626.74609  19622.01563   

      Unnamed: 14  Unnamed: 15  Unnamed: 16  Unnamed: 17  Unnamed: 18  \
1765  73275.07813  73284.82031  73286.98438  73274.98438  73265.07031   
1813  10798.92090  10804.54199  10805.13867  10803.37793  10800.16406   
1355  22163.07031  22162.12891  22156.93359  22158.27148  22153.93555   
1509  29327.03906  29328.96289  29314.17773  29323.99414  29319.62305   
2164  19618.29883  19624.82813  19636.08203  19659.59766  19644.27930   

      Unnamed: 19  Unnamed: 20  Unnamed: 21  Unnamed: 22  Unnamed: 23  \
1765  73293.29688  73261.58594  73271.35938  73278.71875  73274.97656   
1813  10810.75879  10798.35059  10803.24512  10792.21582  10810.55078   
1355  22146.81250  22158.11523  22163.53516  22151.93164  22159.56641   
1509  29314.28906  29329.36523  29317.52344  29327.97852  29313.31641   
2164  19662.24414  19651.03906  19668.10742  19665.88477  19676.67773   

      Unnamed: 24  Unnamed: 25  Unnamed: 26  Unnamed: 27  Unnamed: 28  \
1765  73266.93750  73268.00000  73279.41406  73283.82813  73276.28906   
1813  10795.90039  10796.46484  10804.32324  10804.49219  10804.37109   
1355  22151.34570  22146.88281  22148.27344  22152.25195  22149.99609   
1509  29312.42773  29333.26563  29312.81641  29331.14844  29329.32031   
2164  19677.30273  19690.15820  19687.93555  19694.68945  19693.02148   

      Unnamed: 29  Unnamed: 30  Unnamed: 31  Unnamed: 32  Unnamed: 33  \
1765  73289.00781  73281.08594  73260.36719  73295.69531  73290.46094   
1813  10805.22266  10794.30273  10797.79785  10799.29492  10797.57520   
1355  22148.48242  22149.50000  22155.92969  22156.89063  22147.25586   
1509  29312.52539  29321.88477  29323.76758  29322.71484  29323.73047   
2164  19695.47656  19699.16602  19706.71484  19712.64453  19713.58984   

      Unnamed: 34  Unnamed: 35  Unnamed: 36  Unnamed: 37  Unnamed: 38  \
1765  73289.17969  73288.64844  73280.25781  73294.38281  73303.51563   
1813  10798.26074  10789.30273  10801.98926  10809.45215  10806.40723   
1355  22149.61133  22145.71094  22148.30664  22147.50586  22162.44141   
1509  29311.99023  29310.88281  29317.67188  29316.79883  29339.54297   
2164  19722.04688  19721.89258  19725.70117  19728.70117  19727.10156   

      Unnamed: 39  Unnamed: 40  Unnamed: 41  Unnamed: 42  Unnamed: 43  \
1765  73291.14844  73297.99219  73284.80469  73276.66406  73280.89063   
1813  10801.10059  10808.22754  10806.16309  10804.41016  10789.30859   
1355  22137.21484  22140.06836  22141.81055  22133.78516  22138.91797   
1509  29322.26758  29316.66797  29325.84961  29320.53125  29317.27148   
2164  19737.29883  19729.27539  19741.44336  19741.45508  19752.30273   

      Unnamed: 44  Unnamed: 45  Unnamed: 46  Unnamed: 47  Unnamed: 48  \
1765  73276.23438  73301.77344  73299.10938  73286.22656  73301.37500   
1813  10794.84961  10798.01953  10779.17871  10809.10156  10806.91602   
1355  22139.32422  22144.67578  22145.27539  22138.00000  22141.14648   
1509  29334.04492  29320.50977  29316.20313  29316.76953  29321.56836   
2164  19736.51953  19764.78516  19758.37500  19766.81250  19753.23047   

      Unnamed: 49  Unnamed: 50  Unnamed: 51  Unnamed: 52 

In [14]:
newexo = forest.predict(X_test.tail())

In [15]:
newexo

array([0, 1, 1, 1, 1])